In [19]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.feature import IDF
from pyspark.ml.feature import CountVectorizer

#vectorize tags array for each user
vectorizer = CountVectorizer(inputCol="tokens", outputCol="features").fit(tagsListDF)
countVectors = vectorizer.transform(tagsListDF).select("id", "features")

#find TF-IDF coefficients for each tag
frequencyVectors = countVectors.map(lambda vector: vector[1])
frequencyVectors.cache()
idf = IDF().fit(frequencyVectors)
tfidf = idf.transform(frequencyVectors)

#prepare corpus for LDA
corpus = tfidf.map(lambda x: [1, x]).cache()

#train LDA
ldaModel = LDA.train(corpus, k = 15, maxIterations=100, optimizer="online", docConcentration=2.0, topicConcentration=3.0)

Py4JError: An error occurred while calling o163.__getstate__. Trace:
py4j.Py4JException: Method __getstate__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:272)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:745)



In [6]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

#sc = pyspark.SparkContext('local[*]')
# Load and parse the data
data = sc.textFile("lda_test.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

# Save and load model
ldaModel.save(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")
sameModel = LDAModel\
    .load(sc, "target/org/apache/spark/PythonLatentDirichletAllocationExample/LDAModel")

Learned topics (as distributions over vocab of 11 words):
Topic 0:
 5.52980615098
 7.38108413867
 4.99285303667
 25.3921834865
 7.65002042228
 5.99186860086
 7.68412065218
 1.85886227278
 1.84121713812
 6.31702766636
 11.6375253577
Topic 1:
 9.97423657496
 6.10285729763
 2.55577811354
 7.56157848508
 4.32015299522
 6.87087753383
 16.7146332875
 2.47646500309
 4.59046887497
 9.87018109723
 15.8666187777
Topic 2:
 10.4959572741
 15.5160585637
 4.45136884978
 7.04623802847
 13.0298265825
 9.13725386531
 6.60124606031
 5.66467272413
 1.56831398691
 7.81279123641
 5.49585586457
